In [28]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time


In [29]:

class TireStrategy : 
    def __init__(self) :
        self.S_laps = 0
        self.M_laps = 0
        self.H_laps = 0
        self.time = 0
        self.pitstops = 0
        self.stint = ''

    def __str__(self) :
        return 'Soft Laps: ' + str(self.S_laps) + '\nMeduim Laps: ' + str(self.M_laps) + '\nHard Laps: ' + str(self.H_laps) + '\nTime for Race: ' + toPrettyTime(self.time) + '\nPitstops: ' + str(self.pitstops) + '\nStints: ' + self.stint

def get_best_tire_option(laps, tire_limits = np.array([9, 21])) :
    result = TireStrategy()
    if laps <= tire_limits[0] :
        result.S_laps = laps
        result.time = np.sum(soft(np.arange(1, laps+1)))
    elif(laps <= tire_limits[1]) :
        result.M_laps = laps
        result.time = np.sum(medium(np.arange(1, laps+1)))
    else :
        result.H_laps = laps
        result.time = np.sum(hard(np.arange(1, laps+1)))

    return result

def soft(x):
    return 118 + np.exp(0.33 * x - 2)
def medium(x):
    return 120 + np.exp(0.31 * x - 6.1)
def hard(x):
    return 122 + np.exp(0.3 * x - 9.4)

def toPrettyTime(sek) :
    return str(int(sek / 3600)) + 'h' + str(int((sek % 3600) / 60)) + 'm' + str(int(sek % 60)) + 's'

def readCsv() : 
    df = pd.read_csv('configs.csv')
    return df

def calculate_tire_limit () :
    tire_limit = np.array([0, 0])
    # find oout when medium is faster than soft
    for i in range(1, 100) :
        if medium(i) < soft(i) :
            tire_limit[0] = i
            break
    # find out when hard is faster than medium
    for i in range(1, 100) :
        if hard(i) < medium(i) :
            tire_limit[1] = i
            break

    return tire_limit
    

def find_pit_combinations(laps, pits, bottom_cap=1, top_cap=float('inf')):
    def generate_combinations(current_combination, remaining_laps, remaining_pits, bottom, top):
        if remaining_pits == 0:
            if remaining_laps == 0:
                combinations.append(sorted(current_combination.copy()))
            return
            
        for i in range(bottom, min(remaining_laps, top) + 1):
            current_combination.append(i)
            generate_combinations(current_combination, remaining_laps - i, remaining_pits - 1, i, top)
            current_combination.pop()

    combinations = []
    generate_combinations([], laps, pits, bottom_cap, top_cap)
    return np.unique(np.array(combinations), axis=0)


In [30]:
!pip freeze > requirements.txt

In [9]:
import numpy as np

def soft(x):
    return 117.9 + np.exp(0.23 * x - 1.3)

def medium(x):
    return 120.5 + np.exp(0.169 * x - 3.2)

def hard(x):
    return 121.8 + np.exp(0.092 * x - 2.3)

def total_time(laps, pit_stops, pit_stop_time, lap_time_function, stint_length):
    total_time = 0
    for _ in range(pit_stops):
        total_time += lap_time_function(stint_length) + pit_stop_time
    total_time += lap_time_function(laps - (pit_stops * stint_length))
    return total_time

def find_best_strategy(laps, pit_stop_time, lap_time_functions):
    best_time = float('inf')
    best_strategy = None

    for compound1 in lap_time_functions:
        for compound2 in lap_time_functions:
            for stint_length1 in range(1, laps):
                for stint_length2 in range(1, laps - stint_length1 + 1):
                    pit_stops = laps // stint_length1 - 1
                    total_time_option = total_time(laps, pit_stops, pit_stop_time, compound1, stint_length1) + total_time(stint_length2, 0, pit_stop_time, compound2, stint_length2)

                    if total_time_option < best_time:
                        best_time = total_time_option
                        best_strategy = (compound1, stint_length1, compound2, stint_length2)

    return best_strategy, best_time

laps = 500
pit_stop_time = 30
lap_time_functions = [soft, medium, hard]

best_strategy, best_time = find_best_strategy(laps, pit_stop_time, lap_time_functions)

print("Best strategy:")
print("Tire Compound for First Stint:", best_strategy[0].__name__)
print("Number of Laps for First Stint:", best_strategy[1])
print("Tire Compound for Second Stint:", best_strategy[2].__name__)
print("Number of Laps for Second Stint:", best_strategy[3])
print("Total Time:", best_time)

Best strategy:
Tire Compound for First Stint: hard
Number of Laps for First Stint: 62
Tire Compound for Second Stint: soft
Number of Laps for Second Stint: 1
Total Time: 1556.699295663208


In [32]:
pitstop_time = 30
laps = 0
pits = 0
top_cap = 0
bottom_cap = 0
options = None
strategies = []

tire_limit = calculate_tire_limit()
print (f'Calculated tire limit: {tire_limit[0]} and {tire_limit[1]}')

laps = int(input('Laps: '))
pits = int(input('Pits: '))
if pits == 0 : pits = 0
else : pits += 1
set_caps = input('set Minimum/Maximum Laps with 1 tire? (y/n) ')
set_caps = set_caps.lower()
if set_caps == 'y' :
    bottom_cap = int(input('Minimum Laps: '))
    top_cap = int(input('Maximum Laps: '))
else :
    bottom_cap = 1
    top_cap = laps
    print(f'Using default caps {bottom_cap} and {top_cap}')


print('your configurations:')
print(f'Laps: {laps}')
print(f'Pits: {pits}')
print(f'Bottom cap: {bottom_cap}')
print(f'Top cap: {top_cap}')
print(f'Pitstop time: {pitstop_time}')
print('-----------------------------------------------------------\n')
start_time = time.time()


# -----------------------------------------
# option calclation
# -----------------------------------------
print('now calculating options...')
if(laps // pits <= bottom_cap or laps // pits >= top_cap) :
    print('Laps per stint not capped')
    options = find_pit_combinations(laps, pits, bottom_cap, top_cap)
else :
    print(f'Laps per stint capped at {bottom_cap} and {top_cap}')
    options = find_pit_combinations(laps, pits, bottom_cap, top_cap)

time_to_calculate_options = time.time()
print('-----------------------------------------------------------')
print('calculated ' + str(len(options)) + ' options')
print(f'in {round(time_to_calculate_options-start_time, 3)} seconds ')
print('-----------------------------------------------------------\n')


# -----------------------------------------
# calculating Strategies
# ----------------------------------------- 
print('-----------------------------------------------------------')
print('now calculating Strategies...')
for option in tqdm(options, desc="Processing Options", unit="option"):
    strat = TireStrategy()
    stints = []

    for stint in range(0, len(option)) :
        stints.append(get_best_tire_option(option[stint], tire_limit))

    for stint in range(0, len(stints)) :
        if(stint != 0) :
            strat.time += pitstop_time
            strat.pitstops += 1
        strat.S_laps += stints[stint].S_laps
        strat.M_laps += stints[stint].M_laps
        strat.H_laps += stints[stint].H_laps
        strat.time += stints[stint].time
        if (stints[stint].S_laps != 0) :
            strat.stint += str(f'\nStint {stint+1}: {stints[stint].S_laps} laps with Soft')
        elif (stints[stint].M_laps != 0) :
            strat.stint += str(f'\nStint {stint+1}: {stints[stint].M_laps} laps with Medium')
        else :
            strat.stint += str(f'\nStint {stint+1}: {stints[stint].H_laps} laps with Hard')
        
    strategies.append(strat)

time_to_calculate_strategy = time.time()
print(f'Time to calculate all Strategies {round(time_to_calculate_strategy-time_to_calculate_options, 3)} seconds ')
print('-----------------------------------------------------------\n')    


# -----------------------------------------
# getting best Strategy
# ----------------------------------------- 
print('***********************************************************') 
print(f'total time: {round(time_to_calculate_strategy-start_time, 3)} seconds ')

best = TireStrategy()
besttime = 9999999999999999999999

for strategy in strategies :
    if strategy.time < besttime :
        best = strategy
        besttime = strategy.time


print('Best Stratgy:')
print(str(best))

Calculated tire limit: 9 and 23


KeyboardInterrupt: Interrupted by user

In [37]:
pitstop_time = 30
laps = 0
max_pits = 0
top_cap = 0
bottom_cap = 0
options = None


tire_limit = calculate_tire_limit()
print (f'Calculated tire limit: {tire_limit[0]} and {tire_limit[1]}')

laps = int(input('Laps: '))
max_pits = int(input('Max Pits: '))
if max_pits == 0 : max_pits = 0
else : max_pits += 1
set_caps = input('set Minimum/Maximum Laps with 1 tire? (y/n) ')
set_caps = set_caps.lower()
if set_caps == 'y' :
    bottom_cap = int(input('Minimum Laps: '))
    top_cap = int(input('Maximum Laps: '))
else :
    bottom_cap = 1
    top_cap = laps
    print(f'Using default caps {bottom_cap} and {top_cap}')


print('your configurations:')
print(f'Laps: {laps}')
print(f'Pits: {max_pits-1}')
print(f'Bottom cap: {bottom_cap}')
print(f'Top cap: {top_cap}')
print(f'Pitstop time: {pitstop_time}')
print('-----------------------------------------------------------\n')
start_time = time.time()

besttimes = []

for pits in range(1, max_pits+1) :
    strategies = []
    # print('===========================================================')
    print(f'Pits {pits-1}:')
    # print('===========================================================')
 
    # -----------------------------------------
    # option calclation
    # -----------------------------------------
    # print('now calculating options...')

    if(laps // pits <= bottom_cap or laps // pits >= top_cap) :
        # print('Laps per stint not capped')
        options = find_pit_combinations(laps, pits, 1, laps)
    else :
        # print(f'Laps per stint capped at {bottom_cap} and {top_cap}')

        options = find_pit_combinations(laps, pits, bottom_cap, top_cap)

    time_to_calculate_options = time.time()
    # print('-----------------------------------------------------------')
    # print('calculated ' + str(len(options)) + ' options')
    # print(f'in {round(time_to_calculate_options-start_time, 3)} seconds ')
    # print('-----------------------------------------------------------\n')


    # -----------------------------------------
    # calculating Strategies
    # ----------------------------------------- 
    # print('-----------------------------------------------------------')
    # print('now calculating Strategies...')
    for option in tqdm(options, desc="Processing Options", unit="option"):
        strat = TireStrategy()
        stints = []

        for stint in range(0, len(option)) :
            stints.append(get_best_tire_option(option[stint], tire_limit))

        for stint in range(0, len(stints)) :
            if(stint != 0) :
                strat.time += pitstop_time
                strat.pitstops += 1
            strat.S_laps += stints[stint].S_laps
            strat.M_laps += stints[stint].M_laps
            strat.H_laps += stints[stint].H_laps
            strat.time += stints[stint].time
            if (stints[stint].S_laps != 0) :
                strat.stint += str(f'\nStint {stint+1}: {stints[stint].S_laps} laps with Soft')
            elif (stints[stint].M_laps != 0) :
                strat.stint += str(f'\nStint {stint+1}: {stints[stint].M_laps} laps with Medium')
            else :
                strat.stint += str(f'\nStint {stint+1}: {stints[stint].H_laps} laps with Hard')
            
        strategies.append(strat)

    time_to_calculate_strategy = time.time()
    # print(f'Time to calculate all Strategies {round(time_to_calculate_strategy-time_to_calculate_options, 3)} seconds ')
    # print('-----------------------------------------------------------\n')    


    # -----------------------------------------
    # getting best Strategy
    # ----------------------------------------- 
    best = TireStrategy()
    besttime = 9999999999999999999999

    for strategy in strategies :
        if strategy.time < besttime :
            best = strategy
            besttime = strategy.time


    #print(f'Best Stratgy: with {pits} pits')
    #print(str(best))
    besttimes.append(best)


print('-----------------------------------------------------------')   
print('besttimes:')
for i in range(0, len(besttimes)) :
    print(f'Pits {i}: {toPrettyTime(besttimes[i].time)}')
print('-----------------------------------------------------------')  
print('overall best strategy:')
overall_best = TireStrategy()
overall_best_time = 9999999999999999999999
for i in range(0, len(besttimes)) :
    if besttimes[i].time < overall_best_time :
        overall_best_time = besttimes[i].time
        overall_best = besttimes[i]
print(overall_best) 
print('-----------------------------------------------------------')  
print(f'time to complete: {round(time.time()-start_time, 3)} seconds ')


Calculated tire limit: 9 and 23


your configurations:
Laps: 120
Pits: 6
Bottom cap: 5
Top cap: 45
Pitstop time: 30
-----------------------------------------------------------

Pits 0:


Processing Options: 100%|██████████| 1/1 [00:00<?, ?option/s]


Pits 1:


Processing Options: 100%|██████████| 60/60 [00:00<?, ?option/s]


...

Pits 5:


Processing Options: 100%|██████████| 76668/76668 [00:06<00:00, 11663.76option/s]


Pits 6:


Processing Options: 100%|██████████| 203446/203446 [00:21<00:00, 9262.37option/s] 

-----------------------------------------------------------
besttimes:
Pits 0: 382231306h59m36s
Pits 1: 15h43m3s
Pits 2: 4h7m35s
Pits 3: 4h5m1s
Pits 4: 4h3m38s
Pits 5: 4h2m54s
Pits 6: 4h3m5s
-----------------------------------------------------------
overall best strategy:
Soft Laps: 0
Meduim Laps: 120
Hard Laps: 0
Time for Race: 4h2m54s
Pitstops: 5
Stints: 
Stint 1: 20 laps with Medium
Stint 2: 20 laps with Medium
Stint 3: 20 laps with Medium
Stint 4: 20 laps with Medium
Stint 5: 20 laps with Medium
Stint 6: 20 laps with Medium
-----------------------------------------------------------
time to complete: 32.812 seconds 
